In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import operator
from functools import reduce


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import os 
os.chdir('/Users/deepak/capstone/clutch-baller')
# os.chdir('./clutch-baller')
import practice_code as pc
import indiviual_stats as ind
# os.chdir('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
df = pd.read_csv('/Users/deepak/Desktop/NBA_Data/Every_Season_Data/[2005-2006]-combined-stats.csv')

##Added another column that shows the pts difference between both teams

In [26]:
df = pc.replace_nan(df)

In [28]:
new_df = pc.add_columns(df)

##Remove unnecessary columns

In [31]:
col_to_remove = ['a1', 'a2', 'a3','a4','a5','h1','h2','h3','h4','h5', 'play_length', 'entered', 'left', 'possession', 'shot_distance', 'original_x', 'original_y', 'converted_x', 'converted_y', 'num', 'away', 'home', 'outof', 'opponent', 'reason', 'elapsed', 'play_id']

In [29]:
df_1 = pc.remove_col(new_df, col_to_remove)

##Remove Rows that are not needed (such as fouls, subs, timeouts etc)

In [7]:
rows = ['sub', 'timeout', 'uknown', 'unknown', 'Ejection', 'violat', 'traveling', 'team reb', 'foul',
        'turnover', 'of period', 'lost ball', 'goaltend', 'bad pass', 'illegal', 'jump ball', 'o', 'backcourt']

In [30]:
df_2 = pc.remove_junk_rows(df_1, 'type', rows)

In [32]:
df_2 = pc.add_ids(df_2)

#Make DF for all players stats

In [38]:
player_stats = ind.object_to_int(df_2, 'points')

/Users/deepak/capstone/clutch-baller/indiviual_stats.py:12: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  new_df = df.convert_objects(convert_numeric=True)


In [39]:
assists = ind.assist_stats(player_stats)

In [40]:
blks = ind.blk_stats(player_stats)

In [41]:
names = ind.player_names(player_stats)

In [85]:
full_player_stats = ind.merge_all(names, blks, assists, player_stats)

In [92]:
player_stats_clutch = ind.object_to_int(regular_clutch, 'points')

/Users/deepak/capstone/clutch-baller/indiviual_stats.py:12: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  new_df = df.convert_objects(convert_numeric=True)


In [93]:
assists = ind.assist_stats(player_stats_clutch)

In [94]:
blks = ind.blk_stats(player_stats_clutch)

In [95]:
names = ind.player_names(player_stats_clutch)

In [96]:
full_clutch_stats = ind.merge_all(names, blks, assists, player_stats_clutch)

##Regular Season or playoffs (using df_2)

In [34]:
regular_season, playoffs = pc.regular_or_playoffs(df_2, 'data_set')

##Getting clutch games

In [35]:
regular_clutch = pc.clutch_moment(regular_season)

In [36]:
regular_clutch = pc.remove_not_imp_games(regular_clutch, 'game_id')

##Stats for all players in reg season, playoff, and clutch

In [99]:
full_clutch_stats.head()

,player,total_games,shots_made,shots_missed,FT_made,FT_missed,assist_count,off_rebound,total_rebound,total_blocks
0,Aaron Williams,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Adonal Foyle,13.0,5.0,6.0,2.0,2.0,1.0,6.0,16.0,6.0
2,Adrian Griffin,8.0,0.0,2.0,0.0,2.0,0.0,3.0,9.0,0.0
3,Al Harrington,30.0,14.0,24.0,17.0,11.0,7.0,6.0,18.0,1.0
4,Al Jefferson,4.0,2.0,1.0,0.0,3.0,0.0,1.0,5.0,1.0


In [238]:
full_playoff_stats.head()

,player,total_games,shots_made,shots_missed,FT_made,FT_missed,assist_count,off_rebound,total_rebound,total_blocks,shooting%,FT%,APG,BPG,ORPG,RPG
0,Adrian Griffin,20.0,32.0,27.0,7.0,1.0,24.0,28.0,71.0,2.0,54.2,87.5,1.2,0.1,1.4,3.6
1,Alan Henderson,1.0,0.0,2.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,NaN,0.0,0.0,1.0,2.0
2,Alonzo Mourning,21.0,26.0,11.0,28.0,14.0,3.0,17.0,61.0,24.0,70.3,66.7,0.1,1.1,0.8,2.9
3,Anderson Varejao,13.0,31.0,19.0,26.0,11.0,3.0,18.0,58.0,2.0,62.0,70.3,0.2,0.2,1.4,4.5
4,Andre Miller,5.0,34.0,43.0,14.0,3.0,36.0,8.0,22.0,1.0,44.2,82.4,7.2,0.2,1.6,4.4


In [84]:
full_reg

,player,total_games,shots_made,shots_missed,FT_made,FT_missed,assist_count,off_rebound,total_rebound,total_blocks
0,Adrian Griffin,20.0,32.0,27.0,7.0,1.0,24.0,28.0,71.0,2.0
1,Alan Henderson,1.0,0.0,2.0,0.0,0.0,0.0,1.0,2.0,0.0
2,Alonzo Mourning,21.0,26.0,11.0,28.0,14.0,3.0,17.0,61.0,24.0
3,Anderson Varejao,13.0,31.0,19.0,26.0,11.0,3.0,18.0,58.0,2.0
4,Andre Miller,5.0,34.0,43.0,14.0,3.0,36.0,8.0,22.0,1.0


In [103]:
ids = ind.add_ids(df_2)

In [111]:
full_clutch_stats = ind.merge_ids(full_clutch_stats, ids)

In [115]:
full_clutch_stats = full_clutch_stats.fillna(0)

In [144]:
full_playoff_stats = ind.merge_ids(full_playoff_stats, ids)
full_playoff_stats= full_playoff_stats.fillna(0)


In [121]:
full_reg_stats = ind.merge_ids(full_reg_stats, ids)
full_reg_stats = full_reg_stats.fillna(0)

pandas.core.series.Series

In [130]:
full_reg_stats = ind.get_per_game_stats(full_reg_stats)

In [132]:
full_clutch_stats= ind.get_per_game_stats(full_clutch_stats)

In [133]:
full_playoff_stats= ind.get_per_game_stats(full_playoff_stats)

In [140]:
full_playoff_stats = full_playoff_stats.drop(['id_x', 'id_y', 'id_x'], axis=1)

In [146]:
full_playoff_stats.head()

,player,total_games,shots_made,shots_missed,FT_made,FT_missed,assist_count,off_rebound,total_rebound,total_blocks,shooting%,FT%,APG,BPG,ORPG,RPG,id
0,Adrian Griffin,20.0,32.0,27.0,7.0,1.0,24.0,28.0,71.0,2.0,54.2,87.5,1.2,0.1,1.4,3.6,4.0
1,Alan Henderson,1.0,0.0,2.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,8.0
2,Alonzo Mourning,21.0,26.0,11.0,28.0,14.0,3.0,17.0,61.0,24.0,70.3,66.7,0.1,1.1,0.8,2.9,11.0
3,Anderson Varejao,13.0,31.0,19.0,26.0,11.0,3.0,18.0,58.0,2.0,62.0,70.3,0.2,0.2,1.4,4.5,15.0
4,Andre Miller,5.0,34.0,43.0,14.0,3.0,36.0,8.0,22.0,1.0,44.2,82.4,7.2,0.2,1.6,4.4,19.0


In [147]:
full_reg_stats.head()

,player,total_games,shots_made,shots_missed,FT_made,FT_missed,assist_count,off_rebound,total_rebound,total_blocks,id,shooting%,FT%,APG,BPG,ORPG,RPG
0,Aaron McKie,13.0,3.0,9.0,1.0,1.0,11.0,3.0,20.0,0.0,0.0,25.0,50.0,0.8,0.0,0.2,1.5
1,Aaron Miles,12.0,6.0,12.0,4.0,0.0,24.0,0.0,14.0,1.0,1.0,33.3,100.0,2.0,0.1,0.0,1.2
2,Aaron Williams,43.0,91.0,85.0,40.0,18.0,17.0,64.0,181.0,21.0,2.0,51.7,69.0,0.4,0.5,1.5,4.2
3,Adonal Foyle,75.0,152.0,148.0,41.0,26.0,33.0,143.0,424.0,125.0,3.0,50.7,61.2,0.4,1.7,1.9,5.7
4,Adrian Griffin,50.0,98.0,106.0,41.0,12.0,89.0,75.0,227.0,9.0,4.0,48.0,77.4,1.8,0.2,1.5,4.5


In [148]:
full_clutch_stats.head()

,player,total_games,shots_made,shots_missed,FT_made,FT_missed,assist_count,off_rebound,total_rebound,total_blocks,id,shooting%,FT%,APG,BPG,ORPG,RPG
0,Aaron Williams,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,50.0,NaN,0.0,1.0,0.0,0.0
1,Adonal Foyle,13.0,5.0,6.0,2.0,2.0,1.0,6.0,16.0,6.0,3.0,45.5,50.0,0.1,0.5,0.5,1.2
2,Adrian Griffin,8.0,0.0,2.0,0.0,2.0,0.0,3.0,9.0,0.0,4.0,0.0,0.0,0.0,0.0,0.4,1.1
3,Al Harrington,30.0,14.0,24.0,17.0,11.0,7.0,6.0,18.0,1.0,5.0,36.8,60.7,0.2,0.0,0.2,0.6
4,Al Jefferson,4.0,2.0,1.0,0.0,3.0,0.0,1.0,5.0,1.0,6.0,66.7,0.0,0.0,0.2,0.2,1.2


In [120]:
x = df[(df['event_type']=='end of period') & (df['period']>=4) & (df['pts_difference']!=0)]
x['winner'] = np.where(x.pts_difference >0, 'away', 'home')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [411]:
a = regular_season['player']


In [412]:
b_list = []
for name in a:
    if name not in b_list:
        b_list.append(name)
        

In [574]:
t= get_percent(regular_season, 'player', 'event_type' , b_list)